In [1]:
from pyspark.sql import SparkSession
import mlflow.spark
from mlflow.tracking import MlflowClient

In [2]:
# SparkSession 생성
spark = (SparkSession
         .builder
         .appName("SparkMlflowExampleApp")
         .getOrCreate())

In [3]:
import os
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://192.168.0.14:9000"
os.environ["MLFLOW_TRACKING_URI"] = "http://192.168.0.14:5001"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "miniostorage"

In [5]:
client = MlflowClient()

In [9]:
current_experiment=dict(mlflow.get_experiment_by_name('BOAZ-Test'))
experiment_id=current_experiment['experiment_id']

In [11]:
# 실험 환경 가져오기
experiment = client.get_experiment(experiment_id)
experiment

<Experiment: artifact_location='s3://mlflow/1', creation_time=1680956716995, experiment_id='1', last_update_time=1680956716995, lifecycle_stage='active', name='BOAZ-Test', tags={}>

In [33]:
# 실행 검색
client = MlflowClient()
runs = client.search_runs(experiment_ids = experiment.experiment_id,
                         order_by= ["metrics.rmse"], # rmse가 작은 모델부터 검색
                          max_results = 3)
run_id = runs[0].info.run_id
run_name = runs[0].info.run_name
print(run_id)
print(run_name)
print(runs[0].data.metrics)
print(runs[0].data.params)

47eeb5ab5800448e9646e62d493dbb3e
upset-lynx-684
{'rmse': 234.9094951959742, 'r2': 0.04766391241134604}
{'num_trees': '10', 'max_depth': '5'}


In [52]:
# 모델 로드
pipelineModel = mlflow.spark.load_model(f"runs:/{run_id}/rf")

2023/04/08 13:00:29 INFO mlflow.spark: 'runs:/47eeb5ab5800448e9646e62d493dbb3e/rf1' resolved as 's3://mlflow/1/47eeb5ab5800448e9646e62d493dbb3e/artifacts/rf1'
2023/04/08 13:00:30 INFO mlflow.spark: URI 's3://mlflow/1/47eeb5ab5800448e9646e62d493dbb3e/artifacts/rf1/sparkml' does not point to the current DFS.
2023/04/08 13:00:30 INFO mlflow.spark: File 's3://mlflow/1/47eeb5ab5800448e9646e62d493dbb3e/artifacts/rf1/sparkml' not found on DFS. Will attempt to upload the file.


In [53]:
# 예측
inputDF = spark.read.parquet("./data/sf-airbnb-clean.parquet")
predDF = pipelineModel.transform(inputDF)

In [54]:
predDF.toPandas()[['prediction']].head(3)

,prediction
0,165.077723
1,241.721852
2,109.119017


In [55]:
spark.stop()